In [117]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

from biomart import BiomartServer, BiomartDataset

In [6]:
#with open('evaluation_data/R-HSA-1474290/gene_sets.json') as f:
#    data = json.load(f, object_hook=lambda elem: print(elem))
#wv_from_bin = KeyedVectors.load_word2vec_format("__data/nlp/PubMed-Pyysalo-2013/wikipedia-pubmed-and-PMC-w2v.bin", binary=True)  # C binary format
#wv_from_bin["scna"]

In [73]:
reactome_id = "R-HSA-8982491"
out_dir = "evaluation_data/R-HSA-8982491"

In [118]:
server = BiomartServer("http://www.ensembl.org/biomart")
ds = server.datasets["hsapiens_gene_ensembl"]